### 임포트 라이브러리

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from tensorflow.keras.utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM, Dense

### 코드 분석

#### 전체 코드

In [ ]:
for t in range(len(p)):
    data = np.concatenate([p[t], v[t], s[t]]) # (p_t,v_t;s_t) 꼴의 데이터셋

input_data = np.reshape(data, (1, 1, -1))
listm_units = 50 # 초기 셀의 개수 설정
def initialize_ecm_lstm(input_size, lstm_units):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(None, input_size)))
    model.add(Dense(1))  # 예측할 값이 1차원 (예: 수익률)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
    
centroids = []
centroids.append(data[0])
radiuses = []
radiuses.append(0)

def initial_step(data, centroids, radiuses, model):
    predicted_returns = []

    for t in range(len(data)):
        new_data = np.concatenate([p[t], v[t], s[t]])
        closest_idx = None
        min_distance = float('inf')
        s_min_init = float('inf')

        # 가장 가까운 클러스터 찾기
        for idx, c in enumerate(centroids):
            distance = np.linalg.norm(new_data - c)
            if distance < min_distance:
                min_distance = distance
                closest_idx = idx

        # 클러스터 업데이트
        if closest_idx is not None and min_distance <= radiuses[closest_idx]:
            centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
            radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        else:
                    # 가장 가까운 클러스터 찾기
            for idx, (c, r) in enumerate(centroids, radiuses):
                s_min = np.linalg.norm(new_data - c) + r
                if s_min < s_min_init : 
                    s_min_init = s_min
                    closest_idx = idx

            if closest_idx is not None and s_min > 2 * radiuses[closest_idx]:
                centroids.append(new_data)
                
            else :
                centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
                radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        # LSTM 모델을 사용한 예측
        predicted_return = model.predict(np.reshape(new_data, (1, 1, -1)))
        predicted_returns.append(predicted_return)

    return centroids, radiuses, predicted_returns

### ${p_t} {v_t} {s_t}$ 데이터셋의 형태(행렬구조)의 예시는 아래와 같습니다. -> **티커 별로 가격(종가)데이터와 거래량 데이터 수집할 필요가 있습니다.**

In [ ]:
# 데이터 준비
# 이 부분은 실제 데이터셋에 맞게 조정해야 함
p = np.random.rand(100, 3)  # 가격 데이터
v = np.random.rand(100, 3)  # 거래량 데이터
s = np.random.rand(100, 1)  # 센티먼트 데이터

### 데이터 병합 (${p_t}, {v_t}, {s_t}$)

In [ ]:
for t in range(len(p)):
    data = np.concatenate([p[t], v[t], s[t]]) # (p_t,v_t;s_t) 꼴의 데이터셋

### 입력 데이터 p,v,s를 LSTM에 추가(add)하는 것으로 알고리즘 1의 14 ~ 20(입력층, 출력층, 망각층) 까지 자동으로 구현이 됩니다.
### 이 때, 알고리즘의 소문자 $c_t$는 centroid가 아닌 LSTM Cell

In [ ]:
input_data = np.reshape(data, (1, 1, -1))
listm_units = 50 # 초기 셀의 개수 설정
def initialize_ecm_lstm(input_size, lstm_units):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(None, input_size)))
    model.add(Dense(1))  # 예측할 값이 1차원 (예: 수익률)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### 클러스터링(ECM 부분) 구현. Clustering 패키지를 임포트할 필요가 없이 단순 구현으로 가능한 로직이라고 합니다.

$t=0$일 때의 초기값 정의했으니 라인 15로 이동

In [ ]:
input_data = np.reshape(data, (1, 1, -1))
# LSTM 모델 훈련
model.fit(input_data, s[t], epochs=1, verbose=0)

$t=1$ 일 때부터, $D_{min}$이 정의됩니다. $D_{min} = (||p_{t,k},v_{t,k},s_t) - C_i||)$ -> || || : $Euclidean Distance$

In [ ]:
for t in range(len(p)):
    data = np.concatenate([p[t], v[t], s[t]]) # (p_t,v_t;s_t) 꼴의 데이터셋
    
centroids = []
centroids.append(data[0])
radiuses = []
radiuses.append(0)

def initial_step(data, centroids, radiuses, model):
    predicted_returns = []

    for t in range(len(data)):
        new_data = np.concatenate([p[t], v[t], s[t]])
        closest_idx = None
        min_distance = float('inf')
        s_min_init = float('inf')

        # 가장 가까운 클러스터 찾기
        for idx, c in enumerate(centroids):
            distance = np.linalg.norm(new_data - c)
            if distance < min_distance:
                min_distance = distance
                closest_idx = idx

        # 클러스터 업데이트
        if closest_idx is not None and min_distance <= radiuses[closest_idx]:
            centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
            radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        else:
                    # 가장 가까운 클러스터 찾기
            for idx, (c, r) in enumerate(centroids, radiuses):
                s_min = np.linalg.norm(new_data - c) + r
                if s_min < s_min_init : 
                    s_min_init = s_min
                    closest_idx = idx

            if closest_idx is not None and s_min > 2 * radiuses[closest_idx]:
                centroids.append(new_data)
                
            else :
                centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
                radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        # LSTM 모델을 사용한 예측
        predicted_return = model.predict(np.reshape(new_data, (1, 1, -1)))
        predicted_returns.append(predicted_return)

    return centroids, radiuses, predicted_returns

        